In [1]:
import numpy as np
import exafmm.laplace as laplace

In [2]:
laplace.__doc__

"exafmm's submodule for Laplace kernel"

### 1. create sources and targets

`init_sources()` takes in two arguments: coordinates of source bodies `src_coords` and charges (weights) of source bodies `src_charges`. Both should be `numpy.ndarray`. It returns a list of sources.

`init_targets()` only requires an array of coordinates `trg_coords` from input and returns a list of targets.

In [3]:
nsrcs = 100000
ntrgs = 200000

# generate random positions for particles
src_coords = np.random.random((nsrcs, 3))
trg_coords = np.random.random((ntrgs, 3))

# generate random charges for sources
src_charges = np.random.random(nsrcs)

In [4]:
# create a list of source instances
sources = laplace.init_sources(src_coords, src_charges)

# create a list of target instances
targets = laplace.init_targets(trg_coords)

`sources` and `targets` are two lists, where the type of each element is `exafmm.laplace.Body`.

In [5]:
print(type(sources))
print(type(sources[0]), type(targets[0]))

<class 'list'>
<class 'exafmm.laplace.Body'> <class 'exafmm.laplace.Body'>


### 2. create a LaplaceFmm instance

To run FMM, we need to create a LaplaceFmm instance. The constructor takes in the expansion order `p`, max number of bodies per leaf `ncrit`, and the tree depth `depth`.

For now, the topology of the adaptive tree is only determined by `ncrit`. Changing `depth` does not affect the tree structure. We keep this interface `depth` for future use (a parameter that controls the domain decomposition when MPI is enabled).

In [6]:
p = 10
ncrit = 200
depth = 4
fmm = laplace.LaplaceFmm(p, ncrit, depth)

### 3. setup fmm

Given `sources`, `targets` and `fmm`, the function `setup()` handles three tasks:
- build the tree
- build interaction lists
- pre-compute (or load) invariant matrices.

It returns an octree, whose type is `exafmm.laplace.Tree`.

In [7]:
tree = laplace.setup(sources, targets, fmm)

In [8]:
print(type(tree))

<class 'exafmm.laplace.Tree'>


### 4. evaluate

`evaluate()` triggers the evaluation and returns the potential and gradient as an `numpy.ndarray` of shape `(ntrgs, 4)`.
The $i$-th row of `trg_values` starts with the potential value of the $i$-th target, followed by its three gradient values.

In [9]:
trg_values = laplace.evaluate(tree, fmm)

In [10]:
trg_values[6]

array([ 7222.29226001, -3551.73708416, -6655.99168008, -1981.069423  ])

Set `verbose` to `True` to show timings.

In [13]:
laplace.clear_values(tree)
trg_values = laplace.evaluate(tree, fmm, True)

### 5. check accuracy (optional)

`fmm.verify(tree.leafs)` returns L2-norm the relative error of potential and gradient in a list, compared with the values calculated from direct method. 

In [16]:
fmm.verify(tree.leafs)

[4.207314816559908e-09, 2.712597588585369e-07]

### 6. update charges of sources and run FMM iteratively

In [17]:
niters = 10

for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    
    src_charges = np.random.random(nsrcs)          # generate new random charges
    laplace.update_charges(tree, src_charges)      # update charges
    laplace.clear_values(tree)                     # clear values
    trg_values = laplace.evaluate(tree, fmm)       # evaluate potentials

    print("Error: ", fmm.verify(tree.leafs))       # check accuracy

---------- iteration 0 ----------
Error:  [4.352361023639004e-09, 3.098696646708154e-07]
---------- iteration 1 ----------
Error:  [4.116010536622585e-09, 2.72433242709804e-07]
---------- iteration 2 ----------
Error:  [3.9919809590657845e-09, 2.675022420823116e-07]
---------- iteration 3 ----------
Error:  [4.107537257779539e-09, 2.686511695223002e-07]
---------- iteration 4 ----------
Error:  [4.2748046759563395e-09, 2.7543720602318786e-07]
---------- iteration 5 ----------
Error:  [4.115591889929881e-09, 2.7953515421806347e-07]
---------- iteration 6 ----------
Error:  [4.177888408939821e-09, 2.7484083957097265e-07]
---------- iteration 7 ----------
Error:  [4.125176864100707e-09, 2.772191081921279e-07]
---------- iteration 8 ----------
Error:  [4.262698075852593e-09, 3.0054217106028773e-07]
---------- iteration 9 ----------
Error:  [4.194660853086308e-09, 3.08801226792557e-07]
